In [5]:
import pandas as pd

In [6]:
from recognizer.utils.constants import ROOT_DIR

DATA_DIR = ROOT_DIR / "data" / "data"

In [7]:
with open(DATA_DIR / "preds.jsonl", "r") as f:
    json_lines = f.readlines()

In [8]:
len(json_lines)

68555

In [9]:
json_lines[0]

'{"input":{"userId":"00390c49-c9b8-401a-bf93-738df3316754","itemList":["diarco-jun23","bsf-ribera-ene23","cabal-gastro-jun23","30-macro-carrefour-selecta-abril23","10-patagonia-fudre-may23","karcher-jun23","bsf-dar-ene23","bersa_vea_ene2023","buenosdias-jun23","10-spv-puppis-may233","10-patagonia-chango-jun23","cabal-librerias-jun23","18-macro-premia-junio23","25-macro-changohiper-preferencial-abril23","40-credicoop-unicen-may23","10-santander-yenny-abril23","fiesta-bersa-jun23","fudre-feb23","10-bna-perfumerias-ene23","20-macro-carrefour-carterageneral-abril23","30-patagonia-bk-jun23","20off-bsf-gulliver-ene2023","25-macro-carrefour-preferencial-abril23","30-bbva-peluquerias-abril23","springwall-oct22","yoy-compra-abr23","9-cuotas-nacion-hoteles-ene23","20-itau-avoas-may23","10-ciudad-gastronomia-nov22","cafeterias-abr23","30-credi-supers-jun23","10-spv-cuspide-may233","10-ciudad-fudre-mar23","15-bna-mayoristas-mar23","30off_patagonia_cafeteria_ago2022","bsf_jumbo_ene2023","bica-compr

In [10]:
import json

items = [json.loads(line) for line in json_lines]

In [3]:
{"Item":{"PK":{"S":"evt1000"},"SK":{"S":"--"},"customer_event_time":{"S":"cus1000#2022-11-22 11:22:00"},"record_type":{"S":"event"},"customer_id":{"N":"1000"},"product_id":{"N":"1000"},"event_id":{"N":"1000"},"event_type":{"S":"hover"},"event_time":{"S":"2022-11-22 11:22:00"},"event_status":{"S":"active"},"event_scalar":{"N":"0.5080000"},"description":{"S":"user hovered over target"}}}

{'Item': {'PK': {'S': 'evt1000'},
  'SK': {'S': '--'},
  'customer_event_time': {'S': 'cus1000#2022-11-22 11:22:00'},
  'record_type': {'S': 'event'},
  'customer_id': {'N': '1000'},
  'product_id': {'N': '1000'},
  'event_id': {'N': '1000'},
  'event_type': {'S': 'hover'},
  'event_time': {'S': '2022-11-22 11:22:00'},
  'event_status': {'S': 'active'},
  'event_scalar': {'N': '0.5080000'},
  'description': {'S': 'user hovered over target'}}}

In [18]:
items[0]["input"]["userId"]

'00390c49-c9b8-401a-bf93-738df3316754'

In [20]:
a = zip(items[0]["output"]["recommendedItems"], items[0]["output"]["scores"])

In [29]:
next(a)[1]

0.0035686

In [59]:
import copy

dynamo_items = []

for item in items[:5]:
    print(item)
    user_id = item["input"]["userId"]
    ranked_promos = zip(item["output"]["recommendedItems"], item["output"]["scores"])

    dynamo_item = {
        "Item":{
            "userId":{"S": user_id},
            "items":{"L":[]}
        }
    }
    for promo_slug, score in ranked_promos:
        dynamo_item["Item"]["items"]["L"].append(
            {
                "M": {
                    "itemId": {"S":promo_slug},
                    "score": {"N": score}
                }
            }
        )
    dynamo_items.append(copy.deepcopy(dynamo_item))


{'input': {'userId': '00390c49-c9b8-401a-bf93-738df3316754', 'itemList': ['diarco-jun23', 'bsf-ribera-ene23', 'cabal-gastro-jun23', '30-macro-carrefour-selecta-abril23', '10-patagonia-fudre-may23', 'karcher-jun23', 'bsf-dar-ene23', 'bersa_vea_ene2023', 'buenosdias-jun23', '10-spv-puppis-may233', '10-patagonia-chango-jun23', 'cabal-librerias-jun23', '18-macro-premia-junio23', '25-macro-changohiper-preferencial-abril23', '40-credicoop-unicen-may23', '10-santander-yenny-abril23', 'fiesta-bersa-jun23', 'fudre-feb23', '10-bna-perfumerias-ene23', '20-macro-carrefour-carterageneral-abril23', '30-patagonia-bk-jun23', '20off-bsf-gulliver-ene2023', '25-macro-carrefour-preferencial-abril23', '30-bbva-peluquerias-abril23', 'springwall-oct22', 'yoy-compra-abr23', '9-cuotas-nacion-hoteles-ene23', '20-itau-avoas-may23', '10-ciudad-gastronomia-nov22', 'cafeterias-abr23', '30-credi-supers-jun23', '10-spv-cuspide-may233', '10-ciudad-fudre-mar23', '15-bna-mayoristas-mar23', '30off_patagonia_cafeteria_ago

In [60]:
lines = [json.dumps(dynamo_item).replace(" ", "") for dynamo_item in dynamo_items]

In [61]:
lines[0]

'{"Item":{"userId":{"S":"00390c49-c9b8-401a-bf93-738df3316754"},"items":{"L":[{"M":{"S":{"itemId":"semanamodo-jun23"},"N":{"score":0.4137042}}},{"M":{"S":{"itemId":"farmaciasonline-abr23"},"N":{"score":0.3394863}}},{"M":{"S":{"itemId":"santander-supermiercoles-jun23"},"N":{"score":0.0776529}}},{"M":{"S":{"itemId":"vital-jun23"},"N":{"score":0.0613462}}},{"M":{"S":{"itemId":"cafeterias-abr23"},"N":{"score":0.0567503}}},{"M":{"S":{"itemId":"20off-macro-pagos-jun23"},"N":{"score":0.0192455}}},{"M":{"S":{"itemId":"25-ciudad-cencosud-nov22"},"N":{"score":0.0089104}}},{"M":{"S":{"itemId":"30-patagonia-puppis-nov22"},"N":{"score":0.0035686}}},{"M":{"S":{"itemId":"fudre-feb23"},"N":{"score":0.0020579}}},{"M":{"S":{"itemId":"20-bna-supers-mar23"},"N":{"score":0.0019375}}},{"M":{"S":{"itemId":"primeracompraonline"},"N":{"score":0.0016034}}},{"M":{"S":{"itemId":"20-bbva-farmaonline-abr23"},"N":{"score":0.0013898}}},{"M":{"S":{"itemId":"30-patagonia-bk-jun23"},"N":{"score":0.0009462}}},{"M":{"S":{

In [62]:
with open(DATA_DIR / "parsed.json", "w", encoding="utf-8") as f:
    for line in lines:
        f.write(line + "\n")